In [3]:
import requests
import visualisation
import json


PORT=8000
sim_id = "sim1"
url = f'http://127.0.0.1:{PORT}/control/poll/create_from_example/{sim_id}'
response = requests.post(url)
print(response)